## Module 10 Exercise: Aggregates

Shannon Paylor (sep4hy)

#### Part 1: Function Summaries

* `np.mean` returns the average of all values in an array.

* `np.std` returns the standard deviation of all values in an array.

* `np.var` returns the variance of all values in an array.

* `np.argmin` returns the index of the minimum value of an array.

* `np.argmax` returns the index of the maximum value of an array.

* `np.median` returns the median value of an array.

* `np.percentile` returns the value(s) of an array at one or more specified percentiles. For example, `np.percentile(array, [25, 50, 75])` will return the median and first and third quartiles of a given array.

#### Part 2: Sample Usage of Aggregate Functions

In [ ]:
import numpy as np
import pandas as pd

In [5]:
#create dummy data for testing
dummy = [85, 70, 100, 92]
dummy_2d = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

In [6]:
#median of 1d array
np.median(dummy)

88.5

In [7]:
#mean of 2d array (all values)
np.mean(dummy_2d)

5.0

In [9]:
#mean of 2d array (each row)
np.mean(dummy_2d, axis = 1)

array([2., 5., 8.])

In [26]:
#mean of 2d array (each column)
np.mean(dummy_2d, axis = 0)

array([4., 5., 6.])

In [10]:
#index of max of 1d array
np.argmax(dummy)

2

In [11]:
#get quartiles of 1d array
np.percentile(dummy, [25, 50, 75])

array([81.25, 88.5 , 94.  ])

#### Part 3: Queries on Data

In [14]:
#read in data
#using some census population data downloaded for the project (but doing new queries)
df = pd.read_csv('../project/cbsa_pop_est2019.csv', encoding = 'latin1')

In [15]:
df.head()

,CBSA,MDIV,STCOU,NAME,LSAD,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,...,RESIDUAL2010,RESIDUAL2011,RESIDUAL2012,RESIDUAL2013,RESIDUAL2014,RESIDUAL2015,RESIDUAL2016,RESIDUAL2017,RESIDUAL2018,RESIDUAL2019
0,10180,NaN,NaN,"Abilene, TX",Metropolitan Statistical Area,165252,165252,165585,166634,167442,...,-5,-3,-9,23,-22,-17,-4,-5,-4,-9
1,10180,NaN,48059.0,"Callahan County, TX",County or equivalent,13544,13545,13512,13511,13488,...,-1,-1,-1,-2,-2,-1,0,0,0,-1
2,10180,NaN,48253.0,"Jones County, TX",County or equivalent,20202,20192,20238,20270,19870,...,1,3,14,5,1,1,-1,0,-1,-2
3,10180,NaN,48441.0,"Taylor County, TX",County or equivalent,131506,131515,131835,132853,134084,...,-5,-5,-22,20,-21,-17,-3,-5,-3,-6
4,10420,NaN,NaN,"Akron, OH",Metropolitan Statistical Area,703200,703196,703031,703200,702109,...,-33,-12,-48,-87,-115,-48,-17,-27,-18,-22


In [18]:
#split city+state into individual columns
df[['CITY', 'STATE']] = df['NAME'].str.split(', ', expand = True)

In [21]:
### Query 1 ------------------------------
#how many CBSAs per LSAD type per state?
#CBSA is census bureau statistical area ID
#LSAD is area type ('Metropolitan Statistical Area', 'County or equivalent', 'Metropolitan Division', 'Micropolitan Statistical Area')

df.groupby(['LSAD', 'STATE'])['CBSA'].count()

LSAD                           STATE
County or equivalent           AK        5
                               AL       45
                               AR       41
                               AZ       12
                               CA       45
                                        ..
Micropolitan Statistical Area  WV        4
                               WV-OH     1
                               WV-VA     1
                               WY        6
                               WY-ID     1
Name: CBSA, Length: 220, dtype: int64

Unsurprisingly, Alaska has far fewer statistical areas than California. Interestingly, there are some MSAs that span two states, which makes sense given that some cities near state borders have residents in multiple states.

In [22]:
### Query 2 -------------------------------------------
#"county or equivalent" appears to be a subcategory of MSA
#are population totals grouped by state and LSAD equal for different LSADs?

df.groupby(['STATE', 'LSAD'])['CENSUS2010POP'].sum()

STATE  LSAD                         
AK     County or equivalent              523154
       Metropolitan Statistical Area     478402
       Micropolitan Statistical Area      44752
AL     County or equivalent             4345138
       Metropolitan Statistical Area    3515052
                                         ...   
WV-VA  Micropolitan Statistical Area     114166
WY     County or equivalent              419368
       Metropolitan Statistical Area     167188
       Micropolitan Statistical Area     230886
WY-ID  Micropolitan Statistical Area      31464
Name: CENSUS2010POP, Length: 220, dtype: int64

In [24]:
print("WY MSAs total pop is " + str(167188+230886))

print("AK MSAs total pop is " + str(478402+44752))

523154

It appears that County or equivalent LSADs have the same total population as Metropolitan + Micropolitan Statistical Areas, with some slight discrepancies for MSAs spanning two states, as seen with Wyoming.

In [27]:
### Query 3 ------------------------------
#looking only at MSAs to avoid duplication, what is the total population by city and state in both 2010 and 2019?

df[df['LSAD'].isin(['Metropolitan Statistical Area', 'Micropolitan Statistical Area'])].groupby(['STATE', 'CITY'])[['CENSUS2010POP', 'POPESTIMATE2019']].sum()

CENSUS2010POP  POPESTIMATE2019
STATE CITY                                        
AK    Anchorage            380821           396317
      Fairbanks             97581            96849
      Juneau                31275            31974
      Ketchikan             13477            13901
AL    Albertville           93019            96774
...                           ...              ...
WY    Laramie               36299            38880
      Riverton              40123            39261
      Rock Springs          43806            42343
      Sheridan              29116            30485
WY-ID Jackson               31464            35606

[926 rows x 2 columns]

MSAs should generally match one-to-one with cities, but if there are any cities containing multiple MSAs, we can now see the city-level population data for 2010 compared to 2019.